In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import copy

In [2]:
# 关闭警告
import warnings
warnings.filterwarnings('ignore')

In [3]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super().__init__()
        assert d_model % num_heads == 0
        self.d_model = d_model          # d_model：输入和输出的特征维度，也就是模型的维度。
        self.num_heads = num_heads      # num_heads：注意力头的数量，它用于并行计算多个注意力表示。
        self.d_k = d_model // num_heads # 每个注意力头的维度
        
        self.W_q = nn.Linear(d_model, d_model)  # 查询 权重矩阵
        self.W_k = nn.Linear(d_model, d_model)  # 键 权重矩阵
        self.W_v = nn.Linear(d_model, d_model)  # 值 权重矩阵
        self.W_o = nn.Linear(d_model, d_model)  # 输出 权重矩阵

        self.dropout = nn.Dropout(dropout)      # dropout：dropout 的比率，用于防止过拟合。

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k) # K.transpose(-2, -1)对矩阵K执行转置操作，交换K的倒数第二个维度和最后一个维度。
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9) # 将某些位置的得分（通常是填充位置）设置为非常小的值 -1e9。这是为了在计算注意力分布时，将被掩码的元素的注意力权重压缩到几乎为零，避免影响后续计算。
        attn = F.softmax(scores, dim=-1) # 归一化，得到注意力权重 attn，确保每一行的注意力权重和为1
        attn = self.dropout(attn)
        output = torch.matmul(attn, V) # 加权求和
        return output, attn

    # 实现了 多头注意力机制 中的一部分，即 分割头（split heads） 的操作
    def split_heads(self, x):
        batch_size, seq_len, _ = x.size()  # x 是输入张量：查询（Q）、键（K）或值（V），形状为 (batch_size, seq_len, d_model) 批次的大小、输入序列的长度、输入的特征维度
        return x.view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2) # .view()调整形状并分割为多个头
        # transpose(1, 2)交换第1和第2维度     (batch_size, seq_len, num_heads, d_k) -> (batch_size, num_heads, seq_len, d_k)

    def forward(self, Q, K, V, mask=None):
        # 对查询、键和值分别应用对应的权重矩阵，并通过 split_heads 方法进行分割。此时得到的形状为 (batch_size, num_heads, seq_len, d_k)。
        # 输入 Q、K 和 V 后通过这些线性层进行更新（为每个输入学习一个新的表示！！），这些变换层会在训练过程中通过梯度更新学习到更合适的表示。
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output, attn_weights = self.scaled_dot_product_attention(Q, K, V, mask)

        # .transpose(1, 2)：在多头注意力计算中，为将多个头的输出拼接成一个大的向量（即 d_model），我们需要确保这些头的信息是按每个位置来排列的，而不是按头排列。
        # (batch_size, num_heads, seq_len, d_k) -> (batch_size, seq_len, num_heads, d_k)：通过交换，可以让每个位置（seq_len）具有 num_heads 个头的输出信息。
        # .contiguous(): 在 PyTorch 中，当你对张量进行切片或转置后，该张量可能在内存中不再是连续存储的。调用 .contiguous() 会返回一个在内存中连续存储的新张量。这是为了确保后续调用 .view() 方法不会出错。
        # .view()：attn_output.size(0) 是批次大小 (batch_size)；-1 表示自动推断；这使得每个位置都有一个长度为 d_model 的特征向量（前面操作方便view这一步！！！）
        attn_output = attn_output.transpose(1, 2).contiguous().view(attn_output.size(0), -1, self.d_model)

        # .W_o(attn_output)：attn_output是(batch_size, seq_len, num_heads * d_k)，需将其转换回原始的模型维度 d_model（=num_heads * d_k）
        # 更重要的是：线性变换层 self.W_o 将这些来自不同头的信息统一到一个共同的空间，并对其进行学习，学到如何加权和整合不同头的信息。
        output = self.W_o(attn_output)
        return output


In [4]:
# 由于 Transformer 中没有循环结构（如 RNN 或 LSTM），它无法捕捉序列中单词的顺序信息，因此需要通过位置编码来显式地将序列中的位置信息加入到输入的嵌入（embedding）中
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):  # d_model 是模型的维度; max_len 是位置编码的最大长度（即最大支持的序列长度）
        super().__init__()
        pe = torch.zeros(max_len, d_model)      # 用于存储所有位置的编码信息
        # position：是形状为 (max_len, 1) 的张量，表示每个位置的索引
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)    # unsqueeze(1) 将其形状变为 (max_len, 1)

        # pos：位置索引（表示序列中某个位置）。
        # i：维度索引，( 2i ) 和 ( 2i+1 ) 分别表示偶数和奇数维度————基于偶数和奇数分开，
        # d_{model}：模型的维度（每个词向量的维度）。
        # 10000^{2i / d_{model}}：一个在不同维度上变化的缩放因子，控制不同维度的位置编码的 “ 频率 ” ，或者说频率因子
        # Q：为什么不用2i和2i+1：在计算频率时，所有偶数维度和奇数维度共享相同的频率因子，但它们使用不同的数学函数（正弦和余弦）来编码相同的频率。
        # 正弦和余弦函数的频率因子是相同的，因为它们共享相同的 周期。它们的区别仅仅在于 相位，即它们开始振荡的位置不同。
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        # 为每个位置生成不同频率的正弦和余弦波
        pe[:, 0::2] = torch.sin(position * div_term)  # 0::2 表示从第 0 列开始，每隔一个位置取一个元素
        pe[:, 1::2] = torch.cos(position * div_term)  # 1::2 表示从第 1 列开始，每隔一个位置取一个元素

        # .unsqueeze(0)将 pe 的形状从 (max_len, d_model) 转换为 (1, max_len, d_model)。此时，pe 的第一个维度表示“批量大小”，此处只考虑一个样本，故为1。
        pe = pe.unsqueeze(0)
        # 将 pe 作为一个缓冲区（buffer）注册到模型中。缓冲区（这些张量不参与梯度计算和优化更新），此处pe存储了位置编码，会随模型一起保存和加载
        self.register_buffer('pe', pe)
                     
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]  # 取出 pe 中与输入x序列长度匹配的前 seq_len 个位置(1, seq_len, d_model)，与 x 的形状在维度上匹配。
        # 加法操作不会改变维度，但它使得每个输入元素的表示同时包含了内容信息（由 x 提供）和位置信息（由 pe 提供）。

In [5]:
# 前馈神经网络
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1): # d_model:输入的特征维度 / d_ff:前馈网络隐层的大小，即经过第一层线性变换后特征的维度 / dropout:丢弃比例
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)  # 512、2048
        self.linear2 = nn.Linear(d_ff, d_model)  # 2048、512
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        x = F.relu(self.linear1(x)) # 线性变换后采用 ReLU 激活（带来非线性，使模型能够学习更复杂的函数）
        x = self.dropout(x)         # 随机丢弃
        x = self.linear2(x)         # 恢复到维度 d_model
        return x

In [6]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1): # 输入的特征维度、多头自注意力机制中的头数、前馈神经网络的隐藏层维度、丢弃率
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.ffn = FeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)  # 分别用于自注意力输出和前馈网络输出的正则化
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout) # 在自注意力输出和前馈网络输出后，分别应用 Dropout 操作
        self.dropout2 = nn.Dropout(dropout)
    
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)    # .self_attn(x, x, x, mask)调用多头自注意力，x 被用作查询（Q）、键（K）和值（V），是自注意力机制（Self-Attention）的标准形式
        x = self.norm1(x + self.dropout1(attn_output)) # 残差连接（Dropout + x） + LayerNorm；然后标准化以帮助梯度传播并提高训练的稳定性
        ffn_output = self.ffn(x)                       # 调用ffn：FeedForward 类将执行两次线性变换、ReLU 激活、以及 Dropout
        x = self.norm2(x + self.dropout2(ffn_output))  # 同上
        return x
        

In [7]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        # 自注意力机制层 (Self-Attention)，用于解码器内部的输入间的注意力计算
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)
        # 交叉注意力机制层 (Cross-Attention)，用于解码器中计算目标序列与编码器输出的注意力
        self.cross_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.ffn = FeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)
    
    def forward(self, x, encoder_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)    # tgt_mask 用于遮蔽目标序列中的某些位置，通常用于防止模型看到未来的词（即因果遮蔽）
        x = self.norm1(x + self.dropout1(attn_output))
        # 解码器使用自身的状态去查询和提取与其生成目标相对应的源信息：x 作为查询（Q），而 encoder_output 作为键（K）和值（V）！
        # 交叉注意力 (Cross-Attention)的计算方式与自注意力(Self-Attention)类似，不同的是它的键和值来自编码器的输出，而不是解码器的输入
        # 这允许解码器通过注意力机制聚焦于编码器中的信息，从而在生成目标序列时参考源序列的信息
        # encoder_output 是对源序列的信息进行“检索”的基础，其提供了所有可能需要关注的信息，所以它充当键和值
        attn_output = self.cross_attn(x, encoder_output, encoder_output, src_mask) # src_mask 用于避免编码器（源序列）中的填充部分padding影响解码器的计算
        x = self.norm2(x + self.dropout2(attn_output))
        ffn_output = self.ffn(x)
        x = self.norm3(x + self.dropout3(ffn_output))
        return x

In [8]:
# Transformer架构
class Transformer(nn.Module):
    
    # **src_vocab_size 和 tgt_vocab_size**：源语言和目标语言的词汇表大小。
    # **d_model**：每个词的表示维度，也就是模型的维度，通常在 Transformer 中设置为 512。
    # **num_heads**：每个注意力层的多头数，通常设置为 8。
    # **num_layers**：编码器和解码器中堆叠的层数（论文中是 6 层）。
    # **d_ff**：前馈神经网络的隐藏层维度，通常设置为 2048。
    # **dropout**：Dropout 用于防止过拟合。
    
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=512, num_heads=8, num_layers=6, d_ff=2048, dropout=0.1):
        super().__init__()
        # 将源语言中的每个词映射为一个固定大小的稠密向量
        # src_vocab_size 是源语言的词汇表大小，d_model 是每个词的嵌入向量的维度。nn.Embedding 会根据源语言的词汇表将每个词的索引映射到 d_model 维度的嵌入向量。

        # 假设源语言有 10000 个词汇（src_vocab_size = 10000），并且我们选择 d_model = 512 作为嵌入向量的维度。
        # 那么 nn.Embedding 会创建一个形状为 [10000, 512] 的嵌入矩阵，每一行是一个大小为 512 的词向量。
        # 当输入一个词索引（例如 100），nn.Embedding 会返回该索引对应的 512 维词向量。
        
        self.encoder_embed = nn.Embedding(src_vocab_size, d_model) # 源语言嵌入层（Source Embedding Layer）
        # tgt_vocab_size 是目标语言的词汇表大小，d_model 是嵌入向量的维度。与源语言嵌入层类似，目标语言的每个词也会通过嵌入层映射为 d_model 维的向量
        self.decoder_embed = nn.Embedding(tgt_vocab_size, d_model) # 目标语言嵌入层（Target Embedding Layer）
        self.pos_encoding = PositionalEncoding(d_model)            # 生成和添加位置编码

        # 编码器和解码器层的初始化    N=6 ！
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        # 解码器的每个输出位置都会通过该线性层转化为目标语言中每个词的预测概率分布，从而完成最终的词生成
        # fc：full connected 全连接层！
        self.fc = nn.Linear(d_model, tgt_vocab_size) # 将解码器的输出（维度为 d_model）映射到目标语言的词汇表大小（tgt_vocab_size）
        self.dropout = nn.Dropout(dropout)

    # encode、decode 对过程进一步封装
    def encode(self, src, src_mask):
        src_emb = self.dropout(self.pos_encoding(self.encoder_embed(src)))
        for layer in self.encoder_layers:            # 依次传递给每一层编码器，复用上述定义的 encoder_layers
            src_emb = layer(src_emb, src_mask)
        return src_emb                               # 输出编码后的源语言表示
    
    def decode(self, tgt, encoder_output, src_mask, tgt_mask):
        tgt_emb = self.dropout(self.pos_encoding(self.decoder_embed(tgt)))
        for layer in self.decoder_layers:
            tgt_emb = layer(tgt_emb, encoder_output, src_mask, tgt_mask)
        return tgt_emb                               # 目标语言的解码表示

    # 调用封装好的函数然后进行传播
    def forward(self, src, tgt, src_mask, tgt_mask):
        encoder_output = self.encode(src, src_mask)
        decoder_output = self.decode(tgt, encoder_output, src_mask, tgt_mask)
        return self.fc(decoder_output)

In [9]:
# 生成源序列和目标序列的掩蔽矩阵
# src 是源序列的输入、src_pad_idx 是源序列中为对齐批量数据而填充的无效词索引、tgt 是目标序列的输入、tgt_pad_idx 是目标序列中填充位置的索引
def create_mask(src, tgt, src_pad_idx, tgt_pad_idx):
    # unsqueeze(1).unsqueeze(2)：将 src_mask 从形状 [batch_size, seq_len] 转换为 [batch_size, 1, 1, seq_len]
    # 在计算注意力时需要将 “掩蔽矩阵” 与 “查询、键、值矩阵” 的维度匹配
    src_mask = (src != src_pad_idx).unsqueeze(1).unsqueeze(2) # 比较得到布尔值张量，填充位置为 False，非填充位置为 True
    tgt_mask = (tgt != tgt_pad_idx).unsqueeze(1).unsqueeze(2)
    seq_len = tgt.size(1)  # 获取目标序列 tgt 在第一个维度（即 seq_len）上的大小，也就是目标序列的长度
    # nopeak_mask = (1 - torch.triu(torch.ones(1, seq_len, seq_len), diagonal=1)).bool()
    nopeak_mask = torch.tril(torch.ones(seq_len, seq_len)).bool() # 创建一个大小为 (seq_len, seq_len) 的全 1 张量，提取下三角矩阵（包括主对角线），转换为布尔类型
    tgt_mask = tgt_mask & nopeak_mask.to(tgt.device) # 确保掩蔽矩阵与目标序列的张量在相同的设备上，目标序列中的填充位置和未来位置都将被遮蔽掉
    return src_mask, tgt_mask

In [10]:
# 论文中没有详细列出具体的初始化方法！！
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1: # 检查模型层 m 是否有 weight 属性；> 1排除如 偏置项 这样的单一维度的参数
        nn.init.xavier_uniform_(m.weight.data)      # 使用 Xavier 均匀分布初始化 来初始化该层的权重，m.weight.data访问层的权重张量

# # 初始化一个 Transformer 模型，src_vocab_size 和 tgt_vocab_size 分别是源语言和目标语言的词汇表大小
# model = Transformer(src_vocab_size=10000, tgt_vocab_size=10000)
# # apply 函数会递归地遍历模型中的每一层（即每一个子模块），并将 initialize_weights 函数应用于每一层，对其权重进行 Xavier 初始化
# model.apply(initialize_weights)

## 测试！！！

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [12]:
# ---- 1. 定义模型    ----

In [13]:
# ---- 2. 加载数据集  ----
from datasets import load_dataset

# 加载 WMT 2014 英语到法语数据集
dataset = load_dataset("wmt14", "fr-en")
# 查看数据集的不同部分
print(dataset)  # 输出包含训练、验证和测试集合的信息
# 查看训练集中前几个样本
print(dataset['train'][0])

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/30 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 40836715
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3003
    })
})
{'translation': {'en': 'Resumption of the session', 'fr': 'Reprise de la session'}}


In [14]:
# ---- 3. 配置训练参数 ----
src_pad_idx = tgt_pad_idx = 0
batch_size = 32

In [15]:
from torch.utils.data import DataLoader

# 替换为支持翻译任务的Tokenizer
from transformers import MarianTokenizer
tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

In [47]:
# 处理数据集：分词并将数据集转为 token id
def tokenize_function(examples):
    # 获取源语言（英语）和目标语言（法语）
    source_texts = [example['en'] for example in examples['translation']]
    target_texts = [example['fr'] for example in examples['translation']]
    
    # 对源文本（英语）进行分词
    model_inputs = tokenizer(source_texts, truncation=True, padding="max_length", max_length=64)
    # 对目标文本（法语）进行分词，并将其作为标签
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(target_texts, truncation=True, padding="max_length", max_length=64)
    
    # 将目标文本的 token id 添加到模型输入中作为 labels
    model_inputs["labels"] = labels["input_ids"]
    
    # 转换为 torch.Tensor，确保数据是张量
    model_inputs["input_ids"] = torch.tensor(model_inputs["input_ids"])
    model_inputs["attention_mask"] = torch.tensor(model_inputs["attention_mask"])
    model_inputs["labels"] = torch.tensor(labels["input_ids"])
    return model_inputs

# 选择前 100000 条记录以减少训练时间，您可以根据需求创建较小的数据集
small_train_dataset = dataset['train'].select(range(10000))
small_val_dataset = dataset['validation'].select(range(3000))
small_test_dataset = dataset['test'].select(range(3000))

# 应用分词函数
tokenized_train_datasets = small_train_dataset.map(tokenize_function, batched=True)
tokenized_val_datasets = small_val_dataset.map(tokenize_function, batched=True)
tokenized_test_datasets = small_test_dataset.map(tokenize_function, batched=True)

# tokenized_train_datasets = dataset['train'].map(tokenize_function, batched=True)
# tokenized_val_datasets = dataset['validation'].map(tokenize_function, batched=True)
# tokenized_test_datasets = dataset['test'].map(tokenize_function, batched=True)

# 将数据集格式设置为 PyTorch tensors
tokenized_train_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_val_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_test_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# 创建 DataLoader
train_dataloader = DataLoader(tokenized_train_datasets, batch_size=32, shuffle=True)
val_dataloader = DataLoader(tokenized_val_datasets, batch_size=32)
test_dataloader = DataLoader(tokenized_test_datasets, batch_size=32)

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)  # 打印模型的设备，确保它是在 GPU 上

cuda


In [20]:
# 根据实际词汇表大小初始化模型
model = Transformer(
    src_vocab_size=tokenizer.vocab_size,
    tgt_vocab_size=tokenizer.vocab_size,
    d_model=512,
    num_heads=8,
    num_layers=3
).to(device)
model.apply(initialize_weights)

Transformer(
  (encoder_embed): Embedding(59514, 512)
  (decoder_embed): Embedding(59514, 512)
  (pos_encoding): PositionalEncoding()
  (encoder_layers): ModuleList(
    (0): EncoderLayer(
      (self_attn): MultiHeadAttention(
        (W_q): Linear(in_features=512, out_features=512, bias=True)
        (W_k): Linear(in_features=512, out_features=512, bias=True)
        (W_v): Linear(in_features=512, out_features=512, bias=True)
        (W_o): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ffn): FeedForward(
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (linear2): Linear(in_features=2048, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=

In [21]:
# 检查 train_dataloader 中的 batch 结构 (只需要打印一次)
for batch in train_dataloader:
    print(batch.keys())  # 确认有 input_ids 和 labels 字段
    break  # 只打印第一个 batch，避免输出过多信息

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [22]:
# 打印一些样本
print(tokenized_train_datasets[0])  # 打印第一个样本

{'input_ids': tensor([  660, 10252,   529,  3498,     7,     4,   269,     0, 59513, 59513,
        59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513,
        59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513,
        59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513,
        59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513,
        59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513,
        59513, 59513, 59513, 59513]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'labels': tensor([  660, 14717,     5,     8,   269,     0, 59513, 59513, 59513, 59513,
        59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513,
        59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513,


In [23]:
# # 确保 DataLoader 能正常工作
# for i, batch in enumerate(train_dataloader):
#     print(f"Batch {i + 1}:")
#     print(batch.keys())  # 打印 batch 的键
#     print(f"input_ids shape: {batch['input_ids'].shape}")  # 查看 input_ids 的形状
#     print(f"labels shape: {batch['labels'].shape}")  # 查看 labels 的形状
#     print('-' * 50)
#     if i == 2:  # 查看前 3 个批次
#         break

In [24]:
# 检查tokenizer
# 查看第一个样本的分词结果
sample_text = dataset['train'][0]['translation']['en']  # 获取英文原文
print("Sample text:", sample_text)

# 使用 tokenizer 进行编码
encoded_input = tokenizer(sample_text, truncation=True, padding='max_length', max_length=64)
print("Encoded input:", encoded_input)

Sample text: Resumption of the session
Encoded input: {'input_ids': [660, 10252, 529, 3498, 7, 4, 269, 0, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [25]:
# 假设 train_dataloader 已经定义并加载数据
for i, batch in enumerate(train_dataloader):
    if i < 1:  # 只查看前 1 个批次
        print(f"Batch {i + 1}:")
        print(batch.keys())  # 输出字段名
        # 查看 'input_ids' 和 'labels' 的前几个元素
        print(f"input_ids (first item): {batch['input_ids'][0]}")
        print(f"labels (first item): {batch['labels'][0]}")
        print('-' * 50)
    else:
        break

Batch 1:
dict_keys(['input_ids', 'attention_mask', 'labels'])
input_ids (first item): tensor([  627,     2,    48,    47,    79,  1016,  3425,     2,   192,   314,
        35524,  1729, 15605,   835,    12,    45,   289,     7,  5606,    10,
        23312,     3,     0, 59513, 59513, 59513, 59513, 59513, 59513, 59513,
        59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513,
        59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513,
        59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513,
        59513, 59513, 59513, 59513])
labels (first item): tensor([  679,   157,    19, 23318,    78,   371,  2205, 11319,   283, 39103,
           17,  5088,     8,  4805,    11,    14,     6,  4261,     3,     0,
        59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513,
        59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513,
        59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513, 59513

In [48]:
# Step 4: 训练模型
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm  # 用于进度条显示

# 定义损失函数和优化器
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

# 设置早停参数
early_stopping_patience = 3  # 连续多少个 epoch 没有改进就停止
best_loss = float('inf')
patience_counter = 0

# 假设train_dataloader是焕发过训练集，val_loader是真正验证集
epochs = 100
for epoch in range(epochs):
    model.train()
    total_loss = 0
    
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{epochs}"):
        src = batch['input_ids'].to(device)
        tgt = batch['labels'].to(device)

        # 清空梯度
        optimizer.zero_grad()
        
        # 创建 mask，在实际中可能需要注意 padding token 的索引值。
        src_mask = (src != tokenizer.pad_token_id).unsqueeze(1).unsqueeze(2)    
        tgt_input = tgt[:-1, :]            
        tgt_mask = (tgt != tokenizer.pad_token_id).unsqueeze(1).unsqueeze(2)   
        
        # 前向传播
        outputs = model(src=src, 
                        tgt=tgt,
                        src_mask=src_mask, 
                        tgt_mask=tgt_mask)
        
        outputs_flat = outputs[:, 1:, :].reshape(-1, outputs.shape[-1]) 
        target_flat = tgt[:, 1:].reshape(-1)  
        
        assert outputs_flat.shape[0] == target_flat.shape[0], f"Shape mismatch: {outputs_flat.shape[0]} != {target_flat.shape[0]}"
        
        loss = criterion(outputs_flat, target_flat)  
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()

    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1} - Loss: {average_loss}")
    
    # 验证阶段，计算验证损失以监控性能变化
    model.eval()  # 切换到评估模式
    val_loss = 0
    
    with torch.no_grad():
        for val_batch in val_dataloader:  # 验证数据加载器
            val_src = val_batch['input_ids'].to(device)
            val_tgt = val_batch['labels'].to(device)

            src_mask_val = (val_src != tokenizer.pad_token_id).unsqueeze(1).unsqueeze(2)    
            val_tgt_input = val_tgt[:-1, :]            
            tgt_mask_val = (val_tgt != tokenizer.pad_token_id).unsqueeze(1).unsqueeze(2)   

            val_outputs = model(src=val_src, 
                                tgt=val_tgt,
                                src_mask=src_mask_val, 
                                tgt_mask=tgt_mask_val)
            
            val_outputs_flat = val_outputs[:, 1:, :].reshape(-1, val_outputs.shape[-1]) 
            val_target_flat = val_tgt[:, 1:].reshape(-1)

            assert val_outputs_flat.shape[0] == val_target_flat.shape[0], f"Shape mismatch: {val_outputs_flat.shape[0]} != {val_target_flat.shape[0]}"
            
            val_loss += criterion(val_outputs_flat, val_target_flat).item()

    average_val_loss = val_loss / len(val_dataloader)
    print(f"Validation Loss: {average_val_loss}")

    # Early Stopping Logic
    if average_val_loss < best_loss:
        best_loss = average_val_loss
        patience_counter = 0
        
        # 保存最佳模型参数
        torch.save(model.state_dict(), "transformer_wmt14_fr_en_best.pth")
    else:
        patience_counter += 1
        
    if patience_counter >= early_stopping_patience:
        print(f"Early stopping activated. Stopping training at epoch {epoch + 1}.")
        break

# 保存最终训练好的模型（如果需要）
torch.save(model.state_dict(), "transformer_wmt14_fr_en_final.pth")


Epoch 1/100: 100%|███████████████████████████████████████████████████████████████████| 313/313 [01:23<00:00,  3.76it/s]


Epoch 1 - Loss: 4.500332522316101
Validation Loss: 5.117064080339797


Epoch 2/100: 100%|███████████████████████████████████████████████████████████████████| 313/313 [01:25<00:00,  3.67it/s]


Epoch 2 - Loss: 3.8859990496224106
Validation Loss: 4.943279766021891


Epoch 3/100: 100%|███████████████████████████████████████████████████████████████████| 313/313 [01:24<00:00,  3.69it/s]


...

Epoch 38 - Loss: 0.1950527841147904
Validation Loss: 2.353210730755583


Epoch 39/100: 100%|██████████████████████████████████████████████████████████████████| 313/313 [01:21<00:00,  3.86it/s]


Epoch 39 - Loss: 0.1769103477366816
Validation Loss: 2.3204509539807097


Epoch 40/100: 100%|██████████████████████████████████████████████████████████████████| 313/313 [01:18<00:00,  4.00it/s]


Epoch 40 - Loss: 0.15869916254243913
Validation Loss: 2.3452338011974985
Early stopping activated. Stopping training at epoch 40.


In [49]:
# 确保目标序列中的最大索引
print("Max index in tgt:", max_index)

Max index in tgt: 59513


In [50]:
print("Max index in source input:", src.max())
print("Max index in target input:", tgt.max())

Max index in source input: tensor(59513, device='cuda:0')
Max index in target input: tensor(59513, device='cuda:0')


In [51]:
print(f"Embedding weight shape: {model.encoder_embed.weight.shape}")  # 对于 source embedding
print(f"Embedding weight shape: {model.decoder_embed.weight.shape}")  # 对于 target embedding

Embedding weight shape: torch.Size([59514, 512])
Embedding weight shape: torch.Size([59514, 512])


In [55]:
# Step 5: 加载模型并进行预测

# 加载训练好的模型
model.load_state_dict(torch.load("transformer_wmt14_fr_en.pth"))
model.eval()  # 设置为评估模式

# 定义翻译函数
def translate(sentence, tokenizer, model, device):
    # 对输入句子进行编码
    src = tokenizer.encode(sentence, return_tensors="pt").to(device)
    tgt = torch.ones((1, 1), dtype=torch.long).fill_(tokenizer.pad_token_id).to(device)  # 初始输入为 padding token

    # 生成 src_mask
    src_mask = (src != tokenizer.pad_token_id).unsqueeze(1).unsqueeze(2)  # 生成 src_mask
    # 初始化 tgt_mask (开始时的 tgt_mask 为仅对当前 token 开放)
    tgt_mask = torch.triu(torch.ones(1, 1, 1, 1, device=device), diagonal=1)  # 上三角矩阵

    # print("源句子:", sentence)
    # print("编码后的源:", tokenizer.decode(src[0], skip_special_tokens=True))
    # print("\n翻译过程:\n" + "="*30)

    # 进行推理
    with torch.no_grad():
        for _ in range(64):  # 最多生成64个token
            # 传递 src 和 tgt 以及对应的掩码
            output = model(src, tgt, src_mask=src_mask, tgt_mask=tgt_mask)  
            next_token = output.argmax(dim=-1)[:, -1]  # 获取预测的下一个token
            tgt = torch.cat([tgt, next_token.unsqueeze(0)], dim=1)  # 将预测的 token 添加到 tgt

            # 打印当前预测信息
            # print(f"当前目标序列: {tokenizer.decode(tgt[0], skip_special_tokens=True)}")
            # print(f"预测的下一个 token ID: {next_token.item()}, 对应字符: '{tokenizer.decode(next_token)}'\n")

            # 更新 tgt_mask：为新的生成的 token 创建一个新的目标掩码
            new_tgt_mask = torch.triu(torch.ones(1, 1, tgt.size(1), tgt.size(1), device=device), diagonal=1)
            tgt_mask = new_tgt_mask  # 每次生成新的 tgt_mask 进行替换，不需要拼接
            
            if next_token.item() == tokenizer.eos_token_id:  # 如果预测结束标记，则停止
                # print("生成结束，遇到结束标记。")
                break

    # 解码预测的token id为句子
    translation = tokenizer.decode(tgt[0], skip_special_tokens=True)
    return translation


In [58]:
sentence = "Resumption of the session"
translation = translate(sentence, tokenizer, model, device)
print(f"Original Sentence: {sentence}")
print(f"Translation: {translation}")

Original Sentence: Resumption of the session
Translation: ,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [59]:
# 示例翻译
sentence = "This is a test sentence."
translation = translate(sentence, tokenizer, model, device)
print(f"Original Sentence: {sentence}")
print(f"Translation: {translation}")

Original Sentence: This is a test sentence.
Translation: ,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
